In [4]:
%%bash
# pip install langchain -q
# pip install langchain-community -q
# pip install langchain-huggingface -q
# pip install langgraph -q
# pip install httpx -q
# pip install langchain-google-genai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 26.5 MB/s eta 0:00:00


In [11]:
import os

from langchain_huggingface import HuggingFacePipeline
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain import LLMChain
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

import auth

model_name = 'gemini-1.5-flash' # 'gemini-pro'
llm = ChatGoogleGenerativeAI(model=model_name)

# run the model to try out
print(llm.invoke('The capital of France'))

content='The capital of France is **Paris**. \n' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-4b209b86-37f1-45d8-a940-8c0f80a2bd05-0' usage_metadata={'input_tokens': 5, 'output_tokens': 8, 'total_tokens': 13}


In [6]:
template = """
Question: {question}
Answer: Let's think step by step.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

llm_chain = prompt | llm

question = 'how many planes are landing in frankfurt airport in a day on avearage?'

print(llm_chain.invoke(question))

content='The question cannot be answered from the given source.' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-4472164f-c1e1-4ec7-81db-4860e13cca84-0' usage_metadata={'input_tokens': 32, 'output_tokens': 10, 'total_tokens': 42}


## LangGraph Components

In [8]:
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [12]:
tool = TavilySearchResults(max_result=4)
print(type(tool))
print(tool.name)

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = llm
abot = Agent(model, [tool], system=prompt)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [13]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

result

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'eee29174-ffa7-4fdc-80f8-36434f1d29ce'}
Back to the model!


{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-97701d9f-94d1-4073-a57a-b117a787eba0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'eee29174-ffa7-4fdc-80f8-36434f1d29ce'}], usage_metadata={'input_tokens': 147, 'output_tokens': 22, 'total_tokens': 169}),
  ToolMessage(content='[{\'

In [16]:
result['messages'][-1].content

'The weather in San Francisco is currently partly cloudy with a temperature of 55.9 degrees Fahrenheit. The wind is blowing from the west at 13.6 mph. \n'

In [17]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})
result['messages'][-1].content

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF and LA'}, 'id': '776baef7-f126-4943-8057-7747e669d769'}
Back to the model!


'I am sorry, I cannot answer your question. I can only search for information about current events. \n'

In [20]:
for message in result['messages']:
  print(message.content)
  print('\n')

What is the weather in SF and LA?





[{'url': 'https://www.wunderground.com/hourly/us/ca/base-line/date/2024-6-26', 'content': 'Los Angeles Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Los Angeles area.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Colonia Los Angeles', 'region': 'Francisco Morazan', 'country': 'Honduras', 'lat': 14.05, 'lon': -87.22, 'tz_id': 'America/Tegucigalpa', 'localtime_epoch': 1719399770, 'localtime': '2024-06-26 5:02'}, 'current': {'last_updated_epoch': 1719399600, 'last_updated': '2024-06-26 05:00', 'temp_c': 21.2, 'temp_f': 70.2, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 

In [22]:
# Note, the query was modified to produce more consistent results.
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]

abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024'}, 'id': '359f212f-6808-43f5-958b-28fe12d4c7e9'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri'}, 'id': 'd88914e9-d707-4852-85b0-8e3e42615b61'}
Back to the model!


In [23]:
print(result['messages'][-1].content)

The Kansas City Chiefs won Super Bowl 2024. The Chiefs are headquartered in Missouri. Missouri's GDP in 2022 was $336.63 billion. 

